# Imports

In [1]:
from buspkg.WebDriver import WebDriver
from selenium.webdriver.common.by import By

from concurrent.futures import ThreadPoolExecutor
import requests

# Retrieving files URLs from HTML tag

ANTT provides their data in CSV format through download links in their [website](https://dados.antt.gov.br/dataset/monitriip-servico-regular-viagens).

Each file's URL can be found in a HTML tag with class 'resource-url-analytics'

In [2]:
url = "https://dados.antt.gov.br/dataset/monitriip-servico-regular-viagens"
className = "resource-url-analytics"

driver = WebDriver()
driver.get(url)

elements = driver.find_elements(By.CLASS_NAME, className)
filesURLs = [element.get_attribute("href") for element in elements]

driver.close()

# Filter only CSV files

Among the retrieved URLs, ANTT provides a PDF file with datatypes and a HTML file with a PowerBI about the data

In [3]:
filteredURLs = [url for url in filesURLs if url.endswith('.csv')]

In [4]:
print(len(filteredURLs))

72


# Download CSV files

Download files from filtered URLs using requests and ThreadPools

In [5]:
def DownloadFile(url):
    response = requests.get(url)
    filename = f"../data/rawfiles/{url.split("/")[-1]}"

    with open(filename, mode="wb") as file:
        file.write(response.content)
    
with ThreadPoolExecutor() as executor:
    executor.map(DownloadFile, filteredURLs)